In [1]:
import arcpy

In [2]:
arcpy.env.parallelProcessingFactor = "80%"

In [3]:
arcpy.env.workspace = "C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA.gdb"

In [4]:
import datetime
mydate = datetime.datetime.now()
mydate.strftime("%y%b")

'20Jan'

In [5]:
wdpa_not_comm_name = "wdpa_not_comm_" + mydate.strftime("%y%b")
wdpa_raisg_merge_name = "wdpa_raisg_merge_" + mydate.strftime("%y%b")
mastergrid = "biodiversity_facets_dissolved"

In [6]:
wdpa_not_comm_dis = wdpa_not_comm_name + "_Dis"
wdpa_raisg_merge_dis = wdpa_raisg_merge_name + "_Dis"
wdpa_not_comm_in = wdpa_not_comm_name + "_In"
wdpa_raisg_merge_in = wdpa_raisg_merge_name + "_In"

### Not community WDPA
This layer has a lot of features and repair Geometry is necessary to run anything later

In [7]:
arcpy.RepairGeometry_management(wdpa_not_comm_name)

<Result 'C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA.gdb\\wdpa_not_comm_20Jan'>

In [9]:
#running in arcpy and worked! aafter repair geometry(killed in arcgis pro)
arcpy.analysis.PairwiseDissolve(wdpa_not_comm_name, 
                                wdpa_not_comm_dis, 
                                None, None, "MULTI_PART")

<Result 'C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA.gdb\\wdpa_not_comm_20Jan_Dis'>

In [10]:
#Running in arcpy (start: 7:50 am monday)
arcpy.analysis.PairwiseIntersect([wdpa_not_comm_dis, mastergrid], 
                                 wdpa_not_comm_in, 
                                 "ALL", None, "INPUT")

<Result 'C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA.gdb\\wdpa_not_comm_20Jan_In'>

In [11]:
arcpy.management.AddField(wdpa_not_comm_in, 
                          "AREA_KM2_prot", # the _prot is necessary because  biodiversity_facets has a field called AREA_KM2
                          "DOUBLE", 
                          None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

<Result 'C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA.gdb\\wdpa_not_comm_20Jan_In'>

In [12]:
arcpy.management.CalculateGeometryAttributes(wdpa_not_comm_in, 
                                             "AREA_KM2_prot AREA_GEODESIC", 
                                             '', 
                                             "SQUARE_KILOMETERS", 
                                             "PROJCS['World_Cylindrical_Equal_Area',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Cylindrical_Equal_Area'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],PARAMETER['Standard_Parallel_1',0.0],UNIT['Meter',1.0]]")

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

<Result 'C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA.gdb\\wdpa_not_comm_20Jan_In'>

There was an error, this is a logged bug, the field wasn't calculated. Doing it with python inside arcgis pro, which has worked. 
More information [here](https://community.esri.com/thread/237842-attributeerror-toolvalidator-object-has-no-attribute-islicensed-message)
```python 
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
c:\arcgis\pro\Resources\ArcToolbox\toolboxes\Data Management Tools.tbx#CalculateGeometryAttributes_management.InitializeParameters.py in <module>

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
c:\arcgis\pro\Resources\ArcToolbox\toolboxes\Data Management Tools.tbx#CalculateGeometryAttributes_management.InitializeParameters.py in <module>

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'
```

Because the `biodiversity_facets` layer didn't have the area, I had to calculate it for it and then join using the `CELL_ID`. As this column is now in `biodiversity_facets`, in future intersects the join will not be necessary. 

In [ ]:
arcpy.management.AddField(wdpa_not_comm_in, 
                          "not_comm_prot_prop", 
                          "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

In [ ]:
arcpy.management.CalculateField(wdpa_not_comm_in, 
                                "not_comm_prot_prop", # this cell needs fixing with the right field names
                                "(!wdpa_not_comm_20Jan_In.AREA_KM2! /!cellID_areaKM2.AREA_KM2!) * 100", "PYTHON3", '')

### Community WDPA
This layer has the WDPA community and the RAISG, repair geometry wasn't necesssary.

In [ ]:
#this has worked inside arcgis pro
arcpy.management.Dissolve(wdpa_raisg_merge_name, 
                          wdpa_raisg_merge_dis, 
                          None, None, "MULTI_PART", "DISSOLVE_LINES")

In [10]:
#This has worked in both arcgis pro and arcpy, but I haven't tried it with inputs from made strings
arcpy.analysis.PairwiseIntersect([wdpa_raisg_merge_dis, mastergrid], 
                                 wdpa_raisg_merge_in, 
                                 "ALL", None, "INPUT")

<Result 'C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA.gdb\\wdpa_raisg_merge_20Jan_In2'>

In [ ]:
#This has been ran in arcgis pro, and I have to check if it works with the wdpa_rais_merge_in value
arcpy.management.AddField(wdpa_raisg_merge_in, 
                          "AREA_KM2_prot", 
                          "DOUBLE", 
                          None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

In [ ]:
#This has been ran in arcgis pro, and I have to check if it works with the wdpa_rais_merge_in value
arcpy.management.CalculateGeometryAttributes(wdpa_raisg_merge_in, 
                                             "AREA_KM2_prot AREA_GEODESIC", 
                                             '', 
                                             "SQUARE_KILOMETERS", 
                                             "PROJCS['World_Cylindrical_Equal_Area',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Cylindrical_Equal_Area'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],PARAMETER['Standard_Parallel_1',0.0],UNIT['Meter',1.0]]")

In [ ]:
arcpy.management.AddField(wdpa_raisg_merge_in, 
                          "comm_prot_prop", 
                          "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

In [ ]:
arcpy.management.CalculateField(wdpa_raisg_merge_in, 
                                "comm_prot_prop", # this cell needs fixing with the right field names
                                "(!wdpa_raisg_merge_20Jan_In.AREA_KM2! /!cellID_areaKM2.AREA_KM2!) * 100", "PYTHON3", '')

### Merging Community WDPA and non-Community
A field map needs to be created, check the other notebook
```python
# then create field (double) AREA_KM2_prot DONE
# calculate geometry DONE
# join with the table that has the cell_id and the size of the cell DONE, but no arcpy code because this will not be necessary in the future. 
# create new field (double) "all_prot_prop" DONE
# calculate field DONE
```

In [ ]:
arcpy.management.Merge("wdpa_not_comm_20Jan_In;wdpa_raisg_merge_20Jan_In", 
                       r"C:\Users\Vizuser\Documents\ArcGIS\Projects\WDPA\WDPA.gdb\Intersects_Merge_20Jan", 
                       'FID_wdpa_not_comm_20Jan_Dis "FID_wdpa_not_comm_20Jan_Dis" true true false 4 Long 0 0,First,#,wdpa_not_comm_20Jan_In,FID_wdpa_not_comm_20Jan_Dis,-1,-1;FID_biodiversity_facets_dissolved "FID_biodiversity_facets_dissolved" true true false 4 Long 0 0,First,#,wdpa_not_comm_20Jan_In,FID_biodiversity_facets_dissolved,-1,-1,wdpa_raisg_merge_20Jan_In,FID_biodiversity_facets_dissolved,-1,-1;CELL_ID "CELL_ID" true true false 254 Text 0 0,First,#,wdpa_not_comm_20Jan_In,CELL_ID,0,254,wdpa_raisg_merge_20Jan_In,CELL_ID,0,254;PROP_LAND "MAX_PROP_LAND" true true false 8 Double 0 0,First,#,wdpa_not_comm_20Jan_In,PROP_LAND,-1,-1,wdpa_raisg_merge_20Jan_In,PROP_LAND,-1,-1;ISISLAND "MAX_ISISLAND" true true false 8 Double 0 0,First,#,wdpa_not_comm_20Jan_In,ISISLAND,-1,-1,wdpa_raisg_merge_20Jan_In,ISISLAND,-1,-1;AGRICULTURE "MAX_AGRICULTUR" true true false 8 Double 0 0,First,#,wdpa_not_comm_20Jan_In,AGRICULTURE,-1,-1,wdpa_raisg_merge_20Jan_In,AGRICULTURE,-1,-1;URBAN "MAX_URBAN" true true false 8 Double 0 0,First,#,wdpa_not_comm_20Jan_In,URBAN,-1,-1,wdpa_raisg_merge_20Jan_In,URBAN,-1,-1;RAINFED "MAX_RAINFED" true true false 8 Double 0 0,First,#,wdpa_not_comm_20Jan_In,RAINFED,-1,-1,wdpa_raisg_merge_20Jan_In,RAINFED,-1,-1;AVE_RSR_PC "MAX_AVE_RSR_PC" true true false 8 Double 0 0,First,#,wdpa_not_comm_20Jan_In,AVE_RSR_PC,-1,-1,wdpa_raisg_merge_20Jan_In,AVE_RSR_PC,-1,-1;SR_PC "MAX_SR_PC" true true false 8 Double 0 0,First,#,wdpa_not_comm_20Jan_In,SR_PC,-1,-1,wdpa_raisg_merge_20Jan_In,SR_PC,-1,-1;FOCAL_SPP "FOCAL_SPP" true true false 10000 Text 0 0,First,#,wdpa_not_comm_20Jan_In,FOCAL_SPP,0,10000,wdpa_raisg_merge_20Jan_In,FOCAL_SPP,0,10000;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,wdpa_not_comm_20Jan_In,Shape_Length,-1,-1,wdpa_raisg_merge_20Jan_In,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,wdpa_not_comm_20Jan_In,Shape_Area,-1,-1,wdpa_raisg_merge_20Jan_In,Shape_Area,-1,-1;AREA_KM2 "AREA_KM2" true true false 8 Double 0 0,First,#,wdpa_not_comm_20Jan_In,AREA_KM2,-1,-1,wdpa_raisg_merge_20Jan_In,AREA_KM2,-1,-1;not_comm_prot_prop "not_comm_prot_prop" true true false 8 Double 0 0,First,#,wdpa_not_comm_20Jan_In,not_comm_prot_prop,-1,-1;FID_wdpa_raisg_merge_20Jan_Dis "FID_wdpa_raisg_merge_20Jan_Dis" true true false 4 Long 0 0,First,#,wdpa_raisg_merge_20Jan_In,FID_wdpa_raisg_merge_20Jan_Dis,-1,-1;comm_prot_prop "comm_prot_prop" true true false 8 Double 0 0,First,#,wdpa_raisg_merge_20Jan_In,comm_prot_prop,-1,-1', "NO_SOURCE_INFO")

In [ ]:
arcpy.analysis.PairwiseDissolve("Intersects_Merge_20Jan", 
                                r"C:\Users\Vizuser\Documents\ArcGIS\Projects\WDPA\WDPA.gdb\Intersects_Merge_20Jan_Dis", 
                                "CELL_ID", None, "MULTI_PART")

In [ ]:
arcpy.management.AddField("Intersects_Merge_20Jan_Dis", 
                          "AREA_KM2_prot", 
                          "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

In [ ]:
arcpy.management.CalculateGeometryAttributes("Intersects_Merge_20Jan_Dis", 
                                             "AREA_KM2_prot AREA_GEODESIC", 
                                             '', 
                                             "SQUARE_KILOMETERS", 
                                             "PROJCS['World_Cylindrical_Equal_Area',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Cylindrical_Equal_Area'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],PARAMETER['Standard_Parallel_1',0.0],UNIT['Meter',1.0]]")

In [ ]:
arcpy.management.AddField("Intersects_Merge_20Jan_Dis", 
                          "all_prot_prop", 
                          "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

In [ ]:
arcpy.management.CalculateField("Intersects_Merge_20Jan_Dis", 
                                "Intersects_Merge_20Jan_Dis.all_prot_prop", #this cell needs fixing because it will not be joined. 
                                "(!Intersects_Merge_20Jan_Dis.AREA_KM2_prot! / !cellID_areaKM2.AREA_KM2!) * 100", "PYTHON3", '')

### Getting the information from the 3 tables into one
We can use the simple table with the cellID and the areaKm2 to put there the information. Building the right fieldmap is crucial to also make sure that we have the right colnames in the final table, the one the front end is going to use. 

In [ ]:
arcpy.management.Merge("cellID_areaKM2;Intersects_Merge_20Jan_Dis;wdpa_not_comm_20Jan_In;wdpa_raisg_merge_20Jan_In", 
                       r"C:\Users\Vizuser\Documents\ArcGIS\Projects\WDPA\WDPA.gdb\prot_prop_20Jan", 
                       'CELL_ID "CELL_ID" true true false 254 Text 0 0,First,#,cellID_areaKM2,CELL_ID,0,254,Intersects_Merge_20Jan_Dis,CELL_ID,0,254,wdpa_not_comm_20Jan_In,CELL_ID,0,254,wdpa_raisg_merge_20Jan_In,CELL_ID,0,254;AREA_KM2 "AREA_KM2" true true false 8 Double 0 0,First,#,cellID_areaKM2,AREA_KM2,-1,-1;all_prot_prop "all_prot_prop" true true false 8 Double 0 0,First,#,Intersects_Merge_20Jan_Dis,all_prot_prop,-1,-1;not_comm_prot_prop "not_comm_prot_prop" true true false 8 Double 0 0,First,#,wdpa_not_comm_20Jan_In,not_comm_prot_prop,-1,-1;comm_prot_prop "comm_prot_prop" true true false 8 Double 0 0,First,#,wdpa_raisg_merge_20Jan_In,comm_prot_prop,-1,-1', "NO_SOURCE_INFO")

After merging there will be a lot of `null` values that have to be changed into zeroes. [Here](https://support.esri.com/en/technical-article/000016100) there is a nice techinical article explaining how to do it. `Null` in arcgis is 
```python
None
```

In [13]:
prop_table = 'prot_prop_20Jan'

In [15]:
fieldObs = arcpy.ListFields(prop_table)  
fieldNames = []
for field in fieldObs:
    fieldNames.append(field.name)
del fieldObs
fieldCount = len(fieldNames)

In [18]:
with arcpy.da.UpdateCursor(prop_table, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0"  
        curU.updateRow(rowU)
del curU

The table has to be exported as .csv (use `Table to table`) and then uploaded to the arcgis account see [here](https://developers.arcgis.com/labs/python/import-data/)
Then it has to be "related". The `Join` within arcgis online does not work. 
To create a feature layer that contains related records, you must publish the layer from ArcGIS Pro or ArcMap.

In [ ]:
# again careful with the fieldmaps
arcpy.conversion.TableToTable("prot_prop_20Jan", 
                              r"C:\Users\Vizuser\Documents\ArcGIS\Projects\WDPA", 
                              "prot_prop_20Jan.csv", 
                              '', 
                              r'CELL_ID "CELL_ID" true true false 254 Text 0 0,First,#,C:\Users\Vizuser\Documents\ArcGIS\Projects\WDPA\WDPA.gdb\prot_prop_20Jan,CELL_ID,0,254;AREA_KM2 "AREA_KM2" true true false 8 Double 0 0,First,#,C:\Users\Vizuser\Documents\ArcGIS\Projects\WDPA\WDPA.gdb\prot_prop_20Jan,AREA_KM2,-1,-1;all_prot_p "all_prot_prop" true true false 8 Double 0 0,First,#,C:\Users\Vizuser\Documents\ArcGIS\Projects\WDPA\WDPA.gdb\prot_prop_20Jan,all_prot_prop,-1,-1;not_comm_p "not_comm_prot_prop" true true false 8 Double 0 0,First,#,C:\Users\Vizuser\Documents\ArcGIS\Projects\WDPA\WDPA.gdb\prot_prop_20Jan,not_comm_prot_prop,-1,-1;comm_prot_ "comm_prot_prop" true true false 8 Double 0 0,First,#,C:\Users\Vizuser\Documents\ArcGIS\Projects\WDPA\WDPA.gdb\prot_prop_20Jan,comm_prot_prop,-1,-1', '')